In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display

pd.options.display.float_format = "{:,.2f}".format

In [3]:
user_item_ratings = pd.read_csv(
    "./data/usable_user_item_ratings_prepared.csv.gz", compression="gzip"
)
user_item_ratings

,user_id,item_id,rating
0,U000003,I00037925,0.61
1,U000003,I00189384,0.61
2,U000003,I00256366,0.61
3,U000003,I00267268,0.85
4,U000003,I00298191,0.61
...,...,...,...
1522149,U080024,I01477220,0.61
1522150,U080024,I01526855,0.75
1522151,U080024,I01539849,0.61
1522152,U080024,I01551577,0.75
